In [42]:
'''test api for given hotel'''
from expedia.client import ExpediaClient
import json

# Sofitel Ams, nHow Amsterdam, ClinkNOORD Hostel Ams
property_id = ["22998", "37928040", "10237269"]

# property_id = ["114116366"]    # Sentral Gaslamp Quarter San Diego
checkin = "2025-11-11"
checkout = "2025-11-12"
occupants = "2"
rates = 20

client = ExpediaClient.from_env()

prv_rate = client.fetch_availability(property_id, checkin, checkout, occupants, "priv_pkg", rate_plan_count=rates)
pub_rate = client.fetch_availability(property_id, checkin, checkout, occupants, "pub_prepay", rate_plan_count=rates)
mkt_rate = client.fetch_availability(property_id, checkin, checkout, occupants, "mkt_prepay", rate_plan_count=rates)

print(mkt_rate)

filename = "data/rate_info.json"
with open(filename, "w") as f:
    json.dump(mkt_rate, f, indent=4)  # Pretty-print with indentation

{'37928040': {'property_id': '37928040', 'status': 'available', 'rooms': [{'id': '214543983', 'room_name': 'nhow Premium room Double View with Bath King', 'rates': [{'id': '250227731', 'status': 'available', 'available_rooms': 16, 'refundable': False, 'member_deal_available': False, 'sale_scenario': {'package': True, 'member': False, 'corporate': False, 'distribution': False}, 'merchant_of_record': 'expedia', 'amenities': {'2192': {'id': '2192', 'name': 'Free WiFi'}}, 'links': {'payment_options': {'method': 'GET', 'href': '/v3/properties/37928040/payment-options?token=REhZAQsABAoHQhMHUlNCF1ELUj1DVhdLBQMES1REClxLQg5cRR8WBlIBSxZdCFRpTQxRWVRRB1VXAFYMH1RXBgUbAwNVVx1UVlICSFEBWQIIBAcNAl1WUR8IVURWUAdWRDtZVWgTVgBXRgBccWBiJH0vdENSQkoRXA5WRGsVVhdKX19db1xXWVULBARRA1FVFFZYAFAVVV0HBkwLBgQFH1YDA1cAVlAFBQNVXxJQQUEVX1wHEGkLEwwMTF0fDR0BFUQHRBdfVlY-XV0OAAIBDFgHU1cVBANRDRkEAAoHSFFSWwBJAVZUBVsNUARRAAZUHkdXEUVQUV5GB0UMAhoVUAoUEkcWQkUPXgNDOVtJAEcARQhWAmhbV1NXCV4vcmoneCR5NxJVVQxQE1lNUQFsRAhcVgpWVQENSwdWFAAJM

In [56]:
def rate_has_breakfast(amenities: dict) -> bool:
    for amenity in amenities.values():
        name = str(amenity.get("name", "")).lower()
        if "breakfast" in name:
            return True
    return False

hotel_id = list(mkt_rate.keys())[0]
room_no = 3
room = mkt_rate[hotel_id]['rooms'][room_no]
print(room['room_name'])
for rates in range(0, len(room['rates'])):
    breakfast_inc = rate_has_breakfast(room['rates'][2]["amenities"])
    print(breakfast_inc)

nhow room Double View
True
True
True
True


In [47]:
import expedia.helpers

hotel_id = list(mkt_rate.keys())[0]

df = expedia.helpers.build_rates_dataframe(
    (
        prv_rate[hotel_id],
        pub_rate[hotel_id],  
        mkt_rate[hotel_id] 
    ),
    (
        "priv_rate", 
        "pub_rate", 
        "mkt_rate")
    )

df["discount"] = df["mkt_rate"] - df["priv_rate"]
df["perc_discount"] = df["discount"] / df["mkt_rate"] * 100

df

,room_name,priv_rate,pub_rate,mkt_rate,discount,perc_discount
0,nhow Premium room Double,290.87,341.16,341.16,50.29,14.740884
1,nhow Premium room Double View with Bath King,331.12,388.58,388.57,57.45,14.784981
2,nhow room Double View,272.23,319.18,319.18,46.95,14.709568
3,Room (nhow Mood),307.59,360.82,360.82,53.23,14.752508



1. Make a circle, 
2. Find all hotels within it, 
3. Get both rate types for each
4. Saves as excel

<h3> Step 1: Draw circle</h3>

In [57]:
'''
1. Make a circle, 
2. Find all hotels within it, 
3. Get both rate types for each
4. Saves as excel
'''

import expedia.helpers


checkin = "2025-11-04"
checkout = "2025-11-08"
occupancy_key = '2'

Radius_m = 5000

# RAI Amsterdam
lat = 52.34687283454716
lng = 4.88998731115722

# San Diego
# lat = 32.7057654197668
# lng = -117.16013425869112

# make circle
polygon = expedia.helpers.circle_polygon_geojson(lat, lng, Radius_m, n_points=120)
polygon_string = expedia.helpers.to_geojson_string(polygon)

<H3>Step 2: Get Hotels and details</H3>

In [58]:
import pandas as pd
from expedia.client import ExpediaClient

client = ExpediaClient.from_env()

# get hotels
ids = client.search_geography(
    polygon, 
    include="property_ids", 
    supply_source="expedia"
)
print(f"{len(ids)} properties found")

# Find details on hotels
property_details = {}

# API only takes 250 IDs at once so we batch the calls
# Bind non-ID params up front using partial/lambda
call_property_info = lambda batch: client.fetch_property_summaries(batch)

client.run_batched(
    ids,
    jobs=[(call_property_info, property_details)],
    batch_size=250,
    on_progress=lambda i, n: print(f"Batch {i}/{n}")
)

497 properties found
Batch 1/2
Batch 2/2


In [59]:
# build a dataframe to store all our info
df = pd.DataFrame.from_dict(property_details, orient="index")
room_numbers = []
floor_counts = []
founding_dates = []

for hotel in property_details.values():
    room_numbers.append(hotel.get("statistics", {}).get("52", {}).get("value", ""))
    floor_counts.append(hotel.get("statistics", {}).get("54", {}).get("value", ""))
    founding_dates.append(hotel.get("statistics", {}).get("2515", {}).get("value", ""))


df["rooms"] = room_numbers
# df["floors"] = floor_counts
# df["founding_date"] = founding_dates

In [60]:
df

,property_id,name,address,location,statistics,rooms
22998,22998,Sofitel Legend The Grand Amsterdam,"{'line_1': 'Oudezijds Voorburgwal 197', 'city'...","{'coordinates': {'latitude': 52.371015, 'longi...","{'2515': {'id': '2515', 'name': 'Year Built - ...",178
10237269,10237269,ClinkNOORD - Hostel,"{'line_1': 'Badhuiskade 3', 'city': 'Amsterdam...","{'coordinates': {'latitude': 52.384208, 'longi...","{'2515': {'id': '2515', 'name': 'Year Built - ...",213
21438064,21438064,YAYS Amsterdam East by Numa,"{'line_1': 'Zeeburgerpad 28-29', 'city': 'Amst...","{'coordinates': {'latitude': 52.36696, 'longit...","{'2515': {'id': '2515', 'name': 'Year Built - ...",15
1772121,1772121,Sphinx Hotel,"{'line_1': 'Weteringschans 82', 'city': 'Amste...","{'coordinates': {'latitude': 52.359328, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",22
1524945,1524945,"Max Brown Hotel Museum Square, part of Sircle ...","{'line_1': 'Jan Luykenstraat, 44', 'city': 'Am...","{'coordinates': {'latitude': 52.36, 'longitude...","{'52': {'id': '52', 'name': 'Total number of r...",64
...,...,...,...,...,...,...
32099928,32099928,SUPPER Hotel,"{'line_1': '462 Singel', 'city': 'Amsterdam', ...","{'coordinates': {'latitude': 52.367646, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",27
6041160,6041160,Hotel Titus,"{'line_1': 'Leidsekade 74', 'city': 'Amsterdam...","{'coordinates': {'latitude': 52.365055, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",19
8439319,8439319,Owl Hotel,"{'line_1': 'Roemer Visscherstraat 1', 'city': ...","{'coordinates': {'latitude': 52.361778, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",34
9458607,9458607,Hotel Café Restaurant Abina,"{'line_1': 'Amsterdamseweg 193', 'city': 'Amst...","{'coordinates': {'latitude': 52.311317, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",23


<h3>Step 3: Get rates</h3>

In [61]:
# fetch rates
private_rates, market_rates = {}, {}

call_priv = lambda batch: client.fetch_availability(batch, checkin, checkout, 2, "priv_pkg", rate_plan_count=1)
call_mkt  = lambda batch: client.fetch_availability(batch, checkin, checkout, 2, "mkt_prepay", rate_plan_count=1)

client.run_batched(
    ids,
    jobs=[
        (call_priv, private_rates), 
        (call_mkt, market_rates)
    ],
    batch_size=250,
    on_progress=lambda i, n: print(f"Batch {i}/{n}")
)

# --- extract ---
# Cheapest totals per property for each rate type
priv_map = expedia.helpers.extract_cheapest_rates_by_id(private_rates, occupancy_key, include_room_name=True)
mkt_map  = expedia.helpers.extract_cheapest_rates_by_id(market_rates,  occupancy_key, include_room_name=True)


Batch 1/2
Batch 2/2


<H3>Step 4; Combine Sheet and save</h3>

In [62]:
df["priv_room"] = df["property_id"].map(lambda pid: priv_map.get(pid, (None, None))[0])
df["priv_rate"] = df["property_id"].map(lambda pid: priv_map.get(pid, (None, None))[1])

df["mkt_room"] = df["property_id"].map(lambda pid: mkt_map.get(pid, (None, None))[0])
df["mkt_rate"] = df["property_id"].map(lambda pid: mkt_map.get(pid, (None, None))[1])

df["Delta"] = df["mkt_rate"] - df["priv_rate"]
df["Discount"] = df["Delta"] / df["mkt_rate"] * 100

df.to_excel("data/properties.xlsx")
df

,property_id,name,address,location,statistics,rooms,priv_room,priv_rate,mkt_room,mkt_rate,Delta,Discount
22998,22998,Sofitel Legend The Grand Amsterdam,"{'line_1': 'Oudezijds Voorburgwal 197', 'city'...","{'coordinates': {'latitude': 52.371015, 'longi...","{'2515': {'id': '2515', 'name': 'Year Built - ...",178,"Superior Room, 2 Twin Beds, City View (Garden,...",2441.00,"Superior Room, 2 Twin Beds, City View (Garden,...",2717.71,276.71,10.181734
10237269,10237269,ClinkNOORD - Hostel,"{'line_1': 'Badhuiskade 3', 'city': 'Amsterdam...","{'coordinates': {'latitude': 52.384208, 'longi...","{'2515': {'id': '2515', 'name': 'Year Built - ...",213,"Standard Double Room, Private Bathroom",420.39,"Standard Double Room, Private Bathroom",464.20,43.81,9.437742
21438064,21438064,YAYS Amsterdam East by Numa,"{'line_1': 'Zeeburgerpad 28-29', 'city': 'Amst...","{'coordinates': {'latitude': 52.36696, 'longit...","{'2515': {'id': '2515', 'name': 'Year Built - ...",15,"L 2 Bedroom Apartment, Sofa Bed, Duplex",1174.41,"L 2 Bedroom Apartment, Sofa Bed, Duplex",1423.50,249.09,17.498419
1772121,1772121,Sphinx Hotel,"{'line_1': 'Weteringschans 82', 'city': 'Amste...","{'coordinates': {'latitude': 52.359328, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",22,None,NaN,None,NaN,NaN,NaN
1524945,1524945,"Max Brown Hotel Museum Square, part of Sircle ...","{'line_1': 'Jan Luykenstraat, 44', 'city': 'Am...","{'coordinates': {'latitude': 52.36, 'longitude...","{'52': {'id': '52', 'name': 'Total number of r...",64,Small Room,583.12,Small Room,597.93,14.81,2.476879
...,...,...,...,...,...,...,...,...,...,...,...,...
32099928,32099928,SUPPER Hotel,"{'line_1': '462 Singel', 'city': 'Amsterdam', ...","{'coordinates': {'latitude': 52.367646, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",27,'La Neige' by SUPPER,662.07,'La Neige' by SUPPER,731.09,69.02,9.440698
6041160,6041160,Hotel Titus,"{'line_1': 'Leidsekade 74', 'city': 'Amsterdam...","{'coordinates': {'latitude': 52.365055, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",19,"Basic Room, 2 Twin Beds, Private Bathroom",419.95,"Basic Room, 2 Twin Beds, Private Bathroom",494.37,74.42,15.053502
8439319,8439319,Owl Hotel,"{'line_1': 'Roemer Visscherstraat 1', 'city': ...","{'coordinates': {'latitude': 52.361778, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",34,None,NaN,None,NaN,NaN,NaN
9458607,9458607,Hotel Café Restaurant Abina,"{'line_1': 'Amsterdamseweg 193', 'city': 'Amst...","{'coordinates': {'latitude': 52.311317, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",23,None,NaN,None,NaN,NaN,NaN


In [72]:
coord = (df["location"].iloc[1]['coordinates']['latitude'], df["location"].iloc[1]['coordinates']['longitude'])
coord

(52.384208, 4.903925)